<a href="https://colab.research.google.com/github/mfilipak/AFRAC_IA/blob/main/004_Visualiza%C3%A7%C3%B5es_usando_modelo_pr%C3%A9_treinado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# EM CONSTRUÇÃO

# Experimento 004
##Objetivo: Experimentos de visualização de word embeddings carregando modelo(s) treinado(s) no Experimento 002.
###Descrição: Experimentos iniciais para criação e visualização de word embeddings<br>
Dica: No COLAB Use CTRL SPACE ao invés de TAB para "autocompletar". Ex:pd.re [CTRL SPACE] vai mostrar uma lista contendo as funções e atributos que começam com pd.re (como read_csv, ...) 

In [1]:
from glob import glob
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
import pprint

from gensim.models import Word2Vec
#from sentence_transformers import SentenceTransformer
import joblib
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import seaborn as sns
import pandas as pd
import numpy as np
from mpl_toolkits import mplot3d
import plotly.express as px

In [2]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [3]:
#Copia os dados das NFEs do portal da cidadância pro drive virtual.
import requests  
file_url = "https://raw.githubusercontent.com//mfilipak/AFRAC_IA/main/MODELS/w2vec_pdt_5000"
r = requests.get(file_url, stream = True) 

with open("w2vec_pdt_5000", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block)


In [4]:
# Caso o treino tenha sido feito chamando a classe callback ela precisa ser declarada para permitir a carga

#TO DO: Não foi possível monitorar a loss durante o treino.
from gensim.test.utils import common_texts, get_tmpfile
from gensim.models import Word2Vec
from gensim.models.callbacks import CallbackAny2Vec

#import logging
#logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

class callback(CallbackAny2Vec):
    '''Callback to print loss after each epoch.'''

    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        try:
          loss = model.running_training_loss
          print('Loss after epoch {}: {}'.format(self.epoch, loss))
        except:
          print(f"EXCEPTION - Epoch:{self.epoch}")
        self.epoch += 1



In [5]:
model = Word2Vec.load("w2vec_pdt_5000",  )

In [6]:
list(model.wv.vocab.keys())[:10]

['maza',
 'diluente',
 'p/',
 'epoxi',
 'dp-003',
 '5',
 'l',
 'coletor',
 'mat.perfuro',
 'cortantes']

In [7]:
test_doc = word_tokenize("banana prata".lower())
test_doc_vector = model.infer_vector(test_doc)
test = model.docvecs.most_similar(positive = [test_doc_vector])
test

[(2197, 0.8439711928367615),
 (1069, 0.8090595006942749),
 (4738, 0.8003209829330444),
 (304, 0.7934848666191101),
 (840, 0.7857657670974731),
 (2209, 0.7723360061645508),
 (627, 0.7571239471435547),
 (2478, 0.729112446308136),
 (4466, 0.7227429747581482),
 (833, 0.709032416343689)]

In [8]:
for i, _ in test:
  target =  model.docvecs[i]
  print(i, target.dot(test_doc_vector)/np.linalg.norm(target)/np.linalg.norm(test_doc_vector))

2197 0.8439712
1069 0.8090595
4738 0.800321
304 0.79348475
840 0.78576577
2209 0.772336
627 0.75712395
2478 0.72911245
4466 0.722743
833 0.7090323


In [9]:
#Esse trecho de código copia o text_corpus do jeito que foi feito o treino.
#Posteriormente, vamos fazer com que isso não seja relevante montando all_vectors a partir da inferência
import requests  
file_url = "https://raw.githubusercontent.com//mfilipak/AFRAC_IA/main/DATASET/202201_NFe_NotaFiscalItem.zip"
r = requests.get(file_url, stream = True) 

with open("portal.zip", "wb") as file:  
    for block in r.iter_content(chunk_size = 1024): 
         if block:  
             file.write(block)

DATA_FILE = "portal.zip"
df = pd.read_csv(DATA_FILE, encoding="CP1252",sep=";")
print("O dataframe completo contém:",len(df),"linhas")
df3 = df[['DATA EMISSÃO','DESCRIÇÃO DO PRODUTO/SERVIÇO', 'CÓDIGO NCM/SH', 'CFOP']]
df3.columns = ["DATA", "DESCR", "NCM", "CFOP"]
df3 = df3[df3["NCM"]!=-1] #Filtrando NCMs = -1

text_lengths = np.array([len(_) for _ in df3['DESCR']])
df3 = df3[text_lengths>=3]

#Caso queira eliminar as repetições rodar a linha abaixo
df3 = df3.drop_duplicates(subset=["DESCR"])
random.seed(42)
text_corpus = random.sample(list(df3["DESCR"]),5000)
len(text_corpus)

O dataframe completo contém: 324056 linhas


5000

In [10]:
all_vectors = model.docvecs.vectors_docs

test_doc = word_tokenize("banana prata".lower())
test_doc_vector = model.infer_vector(test_doc)

cos_distances = all_vectors.dot(test_doc_vector)/np.linalg.norm(all_vectors, axis=-1)/np.linalg.norm(test_doc_vector)
for i in np.argsort(cos_distances)[::-1][:10]:
  print(text_corpus[i])


FRUTA IN NATURA BANANA PRATA
OUTROS/BANANA PRATA
BANANA NANICA/CATURRA CLIM PRIM
BANANA ESPECIE COMPRIDA
BANANA NANICA
Banana comprida
MELAO  IN NATURA ESPECIE ORANGE
SAGU PRATA 500GR
DOCE DE BANANA COM 50UND
FLUIDO FREIO DOT4


In [11]:
random.seed(0)
new_text_corpus = random.sample(list(df3["DESCR"]),5000)
len(new_text_corpus)

5000

In [12]:
all_vectors = model.docvecs.vectors_docs

test_doc = word_tokenize("banana prata".lower())
test_doc_vector = model.infer_vector(test_doc)

cos_distances = all_vectors.dot(test_doc_vector)/np.linalg.norm(all_vectors, axis=-1)/np.linalg.norm(test_doc_vector)
for i in np.argsort(cos_distances)[::-1][:10]:
  print(new_text_corpus[i])

IOGURTE NINHO MACA E BANANA 170 G
PLACA LCP RADIO D LAT 2,7MM 6F L: 160032879-025 ANVISA N : 80082910118
ABRAÇADEIRA, MATERIAL PLÁSTICO, COMPRIMENTO TOTAL 110 MM, LARGURA 2,50 MM,APLICAÇÃO
FECH PADO TRIO QUADRA ROS 701-80 MP16500 CR
RABADA
FORMA TUBO PUDIM/BOLO
INTRINSECA | SPRINT: O METODO USADO NO GOOGLE PARA TESTAR E APLICAR NOVAS IDEIAS EM APENAS CINCO DIAS | Ed: 12017
MESA - AUXILIAR
FONTE 24VCC BIVOLT SAIDA DUPLA 3A 72W DIN CERTIFICADA
OUTROS OLEOS LUBRIFICANTES AUTOMOTIVOS                                     - Item 5


In [13]:
stoplist = set('- DE E PARA X COM EM TIPO de A | DO C/ DA / P/ Ed O'.split(' '))
# Lowercase each document, split it by white space and filter out stopwords
texts = [[word for word in document.lower().split() if word not in stoplist]
         for document in new_text_corpus]

# Count word frequencies
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1

# Only keep words that appear more than once
#Ignorei esse filtro pois com poucos exemplos iria cortar quase tudo
processed_corpus = [[token for token in text if frequency[token] > 0] for text in texts]
pprint.pprint(processed_corpus[:5])

[['vavula', 'escape', 'tr4', '2010/...'],
 ['foice', 'rocadeira', 'tipo', 'l,', 'c/', 'cabo,', '1,2m'],
 ['sinapi', '4721', 'brita', '1', '(posto', 'jazida)'],
 ['componente',
  'cortador',
  'legumes,',
  'tipo',
  'manual,',
  'tipo',
  'acessório',
  'lâmina,',
  'material',
  'aço',
  'inoxidável'],
 ['fresa', 'canulada', '06,0', 'mm.', 'rms.80804050147']]


In [14]:
all_vectors = []
for pc in processed_corpus[:]:
  all_vectors += [model.infer_vector(pc)]
all_vectors = np.array(all_vectors)
all_vectors.shape

(5000, 50)

In [15]:
from sklearn.manifold import TSNE
tsne = TSNE(n_components=3, random_state=0)
vectors = tsne.fit_transform(all_vectors)

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,


In [20]:
table = [ [n,*c] for n, c in zip(new_text_corpus,list(vectors)) ]
df = pd.DataFrame(table, columns=["Name", "X", "Y", "Z"])
df

,Name,X,Y,Z
0,VAVULA DE ESCAPE TR4 2010/...,4.595235,2.657425,-9.014860
1,"FOICE ROcADEIRA TIPO L, C/ CABO, 1,2M",-22.145391,4.721488,-0.180204
2,SINAPI 4721 - BRITA 1 (POSTO JAZIDA),19.776888,12.084909,18.585375
3,"COMPONENTE CORTADOR LEGUMES, TIPO MANUAL, TIPO...",-20.518663,24.398233,3.170313
4,"FRESA CANULADA DE 06,0 MM. RMS.80804050147",-4.650954,-21.436649,15.044114
...,...,...,...,...
4995,PINO GUIA PINCA FREIO L200 TRITON/ TR4,5.830154,15.988537,37.441113
4996,FOICE ROCADEIRA COM CABO,28.392593,20.846178,3.791255
4997,HASTE DA CHAVE HEXAGONAL 2.5mm,3.332483,-34.399437,-10.672776
4998,Papel sulfite A4 75g 210mmx297mm PT 500 FL,26.891083,-7.627120,-27.718893


In [52]:
test_doc = word_tokenize("leite em pó".lower())
test_doc_vector = model.infer_vector(test_doc)

cos_distances = all_vectors.dot(test_doc_vector)/np.linalg.norm(all_vectors, axis=-1)/np.linalg.norm(test_doc_vector)
names = [""]*len(new_text_corpus)
for i in np.argsort(cos_distances)[::-1][:10]:
  print(cos_distances[i], new_text_corpus[i])
  names[i] = new_text_corpus[i]


0.96710813 LEITE EM PÓ 200G
0.67915326 Sabão em pó
0.6286573 ACHOCOLATADO EM PÓ
0.5859263 ALFAJOR DOCE LEITE CHOC BR 40G CATANDUVA
0.5592893 PÓ PARA REFRESCO
0.558121 LEITE DE CABRA
0.5562187 CHOC GAROTO BATON LEITE
0.51608694 API 20 E
0.5157831 SULFATO DE ALUMINIO ASPECTO FISICO PO BASEQUIMICA
0.47558418 LEITE L.V.INTEGRAL TERRA VIVA 1LT


In [53]:
fig = px.scatter_3d(df, x="X", y="Y", z="Z", opacity=0.25, text=names, color=names)
fig.update_layout(
    title="DOC EMBEDDINGS IN 3D",
    legend_title="Descrições mais próximas",
    font=dict(
        size=18,
    ))
fig.show()

In [54]:
fig = px.scatter_3d(df.iloc[ np.argsort(cos_distances)[::-1][:10]], x="X", y="Y", z="Z",  text="Name", color="Name")
fig.show()


In [42]:
df

,Name,X,Y,Z
0,VAVULA DE ESCAPE TR4 2010/...,4.595235,2.657425,-9.014860
1,"FOICE ROcADEIRA TIPO L, C/ CABO, 1,2M",-22.145391,4.721488,-0.180204
2,SINAPI 4721 - BRITA 1 (POSTO JAZIDA),19.776888,12.084909,18.585375
3,"COMPONENTE CORTADOR LEGUMES, TIPO MANUAL, TIPO...",-20.518663,24.398233,3.170313
4,"FRESA CANULADA DE 06,0 MM. RMS.80804050147",-4.650954,-21.436649,15.044114
...,...,...,...,...
4995,PINO GUIA PINCA FREIO L200 TRITON/ TR4,5.830154,15.988537,37.441113
4996,FOICE ROCADEIRA COM CABO,28.392593,20.846178,3.791255
4997,HASTE DA CHAVE HEXAGONAL 2.5mm,3.332483,-34.399437,-10.672776
4998,Papel sulfite A4 75g 210mmx297mm PT 500 FL,26.891083,-7.627120,-27.718893
